<a href="https://colab.research.google.com/github/rahiakela/practical-natural-language-processing/blob/chapter-3-text-representation/5_using_pre_trained_word2vec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a pre-trained word2vec model

## Word Embeddings

What does it mean when we say a text representation should capture “distributional similarities between words”? 

Let’s consider some examples. If we’re given the word “USA,” distributionally similar words could be other countries (e.g., Canada, Germany, India, etc.) or cities in the USA. If we’re given the word “beautiful,” words that share some relationship with this word (e.g., synonyms, antonyms) could be considered distributionally similar words. These are words that are likely to occur in similar contexts. 

In 2013, a seminal work by Mikolov et al. showed that their neural network–based word representation model known as “Word2vec,” based on “distributional similarity,” can capture word analogy relationships such as:

`King – Man + Woman ≈ Queen`

While learning such semantically rich relationships, Word2vec ensures that the learned word representations are low dimensional (vectors of dimensions 50–500, instead of several thousands) and dense (that is, most values in these vectors are non-zero). 

Such representations make ML tasks more tractable and efficient. Word2vec led to a lot of work (both pure and applied) in the direction of learning text representations using neural networks. These representations are also called “embeddings.” 

To “derive” the meaning of the word, Word2vec uses distributional similarity and distributional hypothesis. That is, it derives the meaning of a word from its context: words that appear in its neighborhood in the text. So, if two different words (often) occur in similar context, then it’s highly likely that their meanings are also similar. 

Word2vec operationalizes this by projecting the meaning of the words in a vector space where words with similar meanings will tend to cluster together, and words with very different meanings are far from one another.

Conceptually, Word2vec takes a large corpus of text as input and “learns” to represent the words in a common vector space based on the contexts in which they appear in the corpus. 

## Pre-trained word embeddings

Some of the most popular pre-trained embeddings are Word2vec by Google, GloVe by Stanford, and fasttext embeddings by Facebook, to name a few. Further, they’re available for various dimensions like d = 25, 50, 100, 200, 300, 600.

Let us take an example of a pre-trained word2vec model, and how we can use it to look for most similar words. We will use the Google News vectors embeddings. https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM

A few other pre-trained word embedding models, and details on the means to access them through gensim can be found in: https://github.com/RaRe-Technologies/gensim-data

In [1]:
!wget -P /tmp/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-08-28 09:33:59--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.226.91
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.226.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/tmp/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  34.4MB/s    in 46s     

2020-08-28 09:34:45 (34.1 MB/s) - ‘/tmp/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [2]:
#This module ignores the various types of warnings generated
import warnings 
warnings.filterwarnings("ignore") 

#This module provides a way of using operating system dependent functionality
import os 

#This module helps in retrieving information on running processes and system resource utilization
import psutil 
process = psutil.Process(os.getpid())
from psutil import virtual_memory
mem = virtual_memory()

#This module is used to calculate the time
import time 

In [3]:
from gensim.models import Word2Vec, KeyedVectors

pretrainedpath = '/tmp/input/GoogleNews-vectors-negative300.bin.gz'

#Load W2V model. This will take some time, but it is a one time effort! 
pre = process.memory_info().rss
print("Memory used in GB before Loading the Model: %0.2f"%float(pre/(10**9))) #Check memory usage before loading the model
print('-'*10)

start_time = time.time() #Start the timer
ttl = mem.total #Toal memory available

#load the model
w2v_model = KeyedVectors.load_word2vec_format(pretrainedpath, binary=True) 
print("%0.2f seconds taken to load"%float(time.time() - start_time)) #Calculate the total time elapsed since starting the timer
print('-'*10)

print('Finished loading Word2Vec')
print('-'*10)

post = process.memory_info().rss
print("Memory used in GB after Loading the Model: {:.2f}".format(float(post/(10**9)))) #Calculate the memory used after loading the model
print('-'*10)

print("Percentage increase in memory usage: {:.2f}% ".format(float((post/pre)*100))) #Percentage increase in memory after loading the model
print('-'*10)

print("Numver of words in vocablulary: ",len(w2v_model.vocab)) #Number of words in the vocabulary.

Memory used in GB before Loading the Model: 0.16
----------
106.34 seconds taken to load
----------
Finished loading Word2Vec
----------
Memory used in GB after Loading the Model: 5.01
----------
Percentage increase in memory usage: 3100.78% 
----------
Numver of words in vocablulary:  3000000


Here, we find the words that are semantically most similar to the word “beautiful”; the last line returns the embedding vector of the word “beautiful”.

Let us examine the model by knowing what the most similar words are, for a given word!

In [4]:
w2v_model.most_similar('beautiful')

[('gorgeous', 0.8353004455566406),
 ('lovely', 0.810693621635437),
 ('stunningly_beautiful', 0.7329413890838623),
 ('breathtakingly_beautiful', 0.7231341004371643),
 ('wonderful', 0.6854087114334106),
 ('fabulous', 0.6700063943862915),
 ('loveliest', 0.6612576246261597),
 ('prettiest', 0.6595001816749573),
 ('beatiful', 0.6593326330184937),
 ('magnificent', 0.6591402292251587)]

In [5]:
# Let us try with another word! 
w2v_model.most_similar('toronto')

[('montreal', 0.698411226272583),
 ('vancouver', 0.6587257385253906),
 ('nyc', 0.6248831748962402),
 ('alberta', 0.6179691553115845),
 ('boston', 0.611499547958374),
 ('calgary', 0.61032634973526),
 ('edmonton', 0.6100261211395264),
 ('canadian', 0.5944076776504517),
 ('chicago', 0.5911980271339417),
 ('springfield', 0.5888351202011108)]

In [6]:
# What is the vector representation for a word? 
w2v_model['beautiful']

array([-0.01831055,  0.05566406, -0.01153564,  0.07275391,  0.15136719,
       -0.06176758,  0.20605469, -0.15332031, -0.05908203,  0.22851562,
       -0.06445312, -0.22851562, -0.09472656, -0.03344727,  0.24707031,
        0.05541992, -0.00921631,  0.1328125 , -0.15429688,  0.08105469,
       -0.07373047,  0.24316406,  0.12353516, -0.09277344,  0.08203125,
        0.06494141,  0.15722656,  0.11279297, -0.0612793 , -0.296875  ,
       -0.13378906,  0.234375  ,  0.09765625,  0.17773438,  0.06689453,
       -0.27539062,  0.06445312, -0.13867188, -0.08886719,  0.171875  ,
        0.07861328, -0.10058594,  0.23925781,  0.03808594,  0.18652344,
       -0.11279297,  0.22558594,  0.10986328, -0.11865234,  0.02026367,
        0.11376953,  0.09570312,  0.29492188,  0.08251953, -0.05444336,
       -0.0090332 , -0.0625    , -0.17578125, -0.08154297,  0.01062012,
       -0.04736328, -0.08544922, -0.19042969, -0.30273438,  0.07617188,
        0.125     , -0.05932617,  0.03833008, -0.03564453,  0.24

Note that if we search for a word that is not present in the Word2vec model, we’ll see a “key not found” error. 

Hence, as a good coding practice, it’s always advised to first check if the word is present in the model’s vocabulary before attempting to retrieve its vector. 

In [7]:
# What if I am looking for a word that is not in this vocabulary?
w2v_model.most_similar('practicalnlp')

KeyError: ignored

In [8]:
w2v_model['practicalnlp']

KeyError: ignored

In [9]:
w2v_model.most_similar('Ryaan')

KeyError: ignored

Two things to note while using pre-trained models:

1. Tokens/Words are always lowercased. If a word is not in the vocabulary, the model throws an exception.
2. So, it is always a good idea to encapsulate those statements in try/except blocks.

## Getting the embedding representation for full text

We have seen how to get embedding vectors for single words. How do we use them to get such a representation for a full text? A simple way is to just sum or average the embeddings for individual words. We will see an example of this using Word2Vec in Chapter 4. Let us see a small example using another NLP library Spacy - which we saw earlier in Chapter 2 too.

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [18]:
import spacy

# Load the spacy model that we already installed in Chapter 2. This takes a few seconds.
%time nlp = spacy.load('en_core_web_sm')

CPU times: user 693 ms, sys: 137 ms, total: 829 ms
Wall time: 1.01 s


In [19]:
# process a sentence using the model
mydoc = nlp("Canada is a large country")

In [20]:
# Get a vector for individual words
print(mydoc[0].vector)  # vector for 'Canada', the first word in the text 

[ 0.21828675 -1.8616467  -1.8246782   3.9640498   3.1702113   3.4044275
  0.01638174  1.0882976   4.298643    2.6220412   4.9655504  -0.55880976
  0.40846556 -0.5311527  -1.936613    0.4890322   0.01409918  2.017309
  1.9619753  -0.3345103  -2.0549996  -1.9780366   1.4000814  -3.9780545
 -1.4757934  -0.0758431  -0.15845299 -2.4204073  -0.22936638 -2.2050803
 -0.3578331  -1.9166979  -1.1512874  -2.2362876   3.0028348  -2.8118327
  4.337387    0.9023212  -1.3791567   1.4033097   0.36686432  1.2876883
 -0.96107507 -4.0383596  -2.529714    1.3005439  -0.3787139  -0.9970173
  0.580034    3.9643373  -0.5534672  -1.7696033  -1.928772   -1.1359324
 -4.5521493   2.0064287   3.4537764   0.8355992   2.3804865  -2.7131867
 -1.2354802  -0.41219887 -0.83612657  0.42878282  3.4744697  -0.57708937
  2.292963   -4.568878    1.1227657   1.0749986  -3.4492185   1.0809329
 -1.3786955   0.21817788  1.15269    -0.93688565  1.9326314  -2.3508701
 -4.635231    0.07333571  1.2330925  -2.1642141  -1.450435   -1

In [21]:
# Averaged vector for the entire sentence
print(mydoc.vector)

[-0.41918245  0.16200495 -0.79129374  1.4938685   1.3054081   2.0170395
  1.5175831   0.93228465  2.1891901   1.4416349   0.06106711  0.09743678
 -0.2257375   0.17713971 -1.5512953   0.2645687  -1.0485003  -1.080864
 -0.8598553  -1.0081775   0.6843368  -0.6533736  -0.32018867 -0.45141286
 -1.5463241   0.8096622   0.66305393 -1.3783945   0.9841442  -0.6920743
  0.22289622  0.15090446 -1.1180314  -1.9345913   0.38503057 -1.8199227
  1.3990417  -1.0615559  -1.9546788  -0.2100529   1.9202824  -0.3749935
 -0.25492477 -2.1416779  -0.6990263  -0.03200452 -0.9764668   1.7387159
 -0.14891338  2.310504    2.7912867  -1.0199782   0.10864041  0.5835435
 -3.1004105   1.5820146   2.0926924  -0.40339088 -0.5991646  -1.3402617
 -0.7594353   0.2780761   2.629024    0.21917906  1.6852343   0.01221395
  1.06303    -2.0580707   0.38551608 -1.1065528  -1.5958662  -0.6947671
 -1.4428566  -0.38099432  0.50667036  0.4246232   1.2565958  -0.09830198
 -1.0119321  -0.9477395   1.7218593  -1.3920348   0.73801583 

In [22]:
# What happens when I give a sentence with strange words (and stop words), and try to get its word vector in Spacy?
temp = nlp("practicalnlp is a newword")
temp[0].vector

array([ 1.4743975 , -0.9622246 , -1.1067446 ,  0.71745956,  3.6869755 ,
       -1.4803706 ,  2.6486013 , -0.02807039,  2.0256255 ,  3.9974196 ,
        2.4013276 ,  2.8038695 ,  2.1188593 , -1.0725985 , -1.8718748 ,
       -1.7074881 , -0.47109914,  1.753031  , -2.5303397 , -0.6910662 ,
        1.4618394 ,  2.451487  , -2.729975  , -1.2108035 , -1.0596836 ,
       -0.86415946, -1.8492069 , -1.3960482 ,  0.9203042 , -1.0206674 ,
        2.9118538 , -1.1872265 ,  0.1711235 , -3.0739012 ,  1.3036946 ,
       -2.8744037 ,  4.8433757 ,  0.5957062 , -2.63268   ,  1.5330828 ,
        3.3766036 ,  2.9181588 , -1.454087  , -1.4249781 , -1.578454  ,
        1.8532394 , -1.0139503 , -0.20046425, -0.5760678 ,  1.9376096 ,
       -0.3732175 , -1.9566089 , -1.7265924 , -0.9403594 , -0.6440577 ,
        1.1401592 ,  2.6202524 ,  0.08162385,  1.40631   ,  1.4846356 ,
       -1.5503293 , -4.1603303 ,  0.78613114,  1.4033163 ,  2.3532844 ,
       -0.48720837,  2.444285  , -3.873241  ,  2.10482   ,  2.81

In [23]:
temp = nlp("Ryaan is a king")
temp[0].vector

array([ 7.4579835e-02,  4.7708580e-01, -6.1412704e-01,  3.5218694e+00,
        2.9170876e+00,  2.5409222e+00,  8.9063656e-01,  9.9246132e-01,
        2.6454415e+00,  2.6988585e+00,  1.2484885e+00, -1.4995390e+00,
        1.6782302e+00,  1.3731290e+00, -3.4341879e+00,  2.5433052e+00,
        9.5429623e-01,  3.4910271e+00,  2.8027713e+00, -9.9670774e-01,
        2.5236878e-01, -9.4356346e-01, -7.7272609e-02, -2.8652625e+00,
       -8.0851555e-01, -1.2593380e+00, -2.4018660e+00, -1.9876711e+00,
        1.3499756e+00, -1.4541742e+00,  1.8580415e+00, -1.3030741e+00,
        7.9284292e-01, -1.7005992e+00,  3.2878261e+00, -8.0671114e-01,
        6.7371144e+00, -1.1776235e+00, -1.5224946e+00, -4.1435361e-03,
        1.6483667e+00,  1.0591946e+00, -8.0203593e-01, -3.7307553e+00,
       -1.4754515e+00,  8.6113429e-01, -1.8348861e+00,  1.8509170e-01,
       -1.8935969e-01,  3.7280464e+00,  1.1278170e-01, -2.4657631e+00,
       -4.4710441e+00, -1.3550012e+00, -6.8296833e+00,  2.2718692e+00,
      